# Point Process

In [9]:
import pandas as pd
import numpy as np
import theano
import pymc3 as pm
from scipy.stats import ks_2samp
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_context('notebook')
plt.style.use('seaborn-darkgrid')
print('Running on PyMC3 v{}'.format(pm.__version__))
#theano.config.gcc.cxxflags = "-Wno-c++11-narrowing"
import warnings

Running on PyMC3 v3.8


## Dirichlet Mixture Model

In [10]:
def Bayesian_mixture_model(ISI_data):
    
    with pm.Model() as model:
        
        BoundedNormal = pm.Bound(pm.Normal, lower=0.0)
        mu1 = pm.Uniform('mu1',lower=0.01,upper=0.1)
        lam1 = pm.Uniform('lam1',lower=0.01,upper=0.04)
        obs1 = pm.Wald.dist(mu=mu1,lam=lam1)


        mu2 = pm.Uniform('mu2',lower=0,upper=0.3)
        sigma = pm.Uniform('sigma',lower=0.0001,upper=0.5)
        obs2 = pm.Bound(pm.Normal, lower=0.0).dist(mu=mu2,sigma=sigma)

        mu3 = pm.Uniform('mu3',lower=0.1,upper=0.6)
        sigma3 = pm.Uniform('sigma3',lower=0.0001,upper=0.5)
        obs3 = pm.Bound(pm.Normal, lower=0.0).dist(mu=mu3,sigma=sigma3)


        w = pm.Dirichlet('w', a=np.array([1., 1., 1.]))

        like = pm.Mixture('like', w=w, comp_dists = [obs1, obs2, obs3], observed=ISI_data)


        step = pm.Metropolis(target_accept=0.9)
        trace = pm.sample(step=step,draws=500,tune=100,progressbar=False,random_seed=10,cores=6)
     

    map_estimate = pm.find_MAP(model=model)
    del map_estimate['w_stickbreaking__']
    del map_estimate['mu1_interval__']
    del map_estimate['lam1_interval__']
    del map_estimate['mu2_interval__']
    del map_estimate['sigma_interval__']
    del map_estimate['mu3_interval__']
    del map_estimate['sigma3_interval__']
    
    map_estimate['w1'] = map_estimate['w'][0]
    map_estimate['w2'] = map_estimate['w'][1]
    map_estimate['w3'] = map_estimate['w'][2]
    
    del map_estimate['w']
    
    map_estimate['n_spikes'] = ISI_data.shape[0]
    
    
    with model:
        ppc_trace = pm.sample_posterior_predictive(trace, 50,model=model,progressbar=False,random_seed=10)
    
    lista_samples=[]
    for i in list(ppc_trace['like']):
        lista_samples.extend(i)
    
    
    print('P_value: ',ks_2samp(lista_samples,ISI_data,mode = 'asymp').pvalue)
    
    return map_estimate

## Inverse Gaussian

In [11]:
def Inverse_Gaussian_model(ISI_data):    
    
    with pm.Model() as model:
        BoundedNormal = pm.Bound(pm.Normal, lower=0.0)
        mu1 = pm.Uniform('mu1',lower=0.01,upper=0.1)
        lam1 = pm.Uniform('lam1',lower=0.01,upper=0.04)
        obs1 = pm.Wald('like',mu=mu1,lam=lam1,observed=ISI_healthy)


        step = pm.NUTS(target_accept=0.9)
        trace = pm.sample(step=step,draws=1000,tune=1000,cores=6)

        map_estimate = pm.find_MAP(model=model)
        
    
    del map_estimate['mu1_interval__']
    del map_estimate['lam1_interval__']
    
    
    
    
    map_estimate['n_spikes'] = ISI_data.shape[0]
    
    
    with model:
        ppc_trace = pm.sample_posterior_predictive(trace, 200,model=model)
    
    lista_samples=[]
    for i in list(ppc_trace['like']):
        lista_samples.extend(i)
    
    
    print('P_value: ',ks_2samp(lista_samples,ISI_data,mode = 'asymp').pvalue)
    
    return map_estimate

## Import data

In [12]:
data_healthy = np.genfromtxt('Data after SS/2019-01-23T11-05-52MIP3 healthy cortical .h5.txt', delimiter=',')

## Consider only neurons with more than 1000 spikes

In [13]:
list_healthy_neuron = []
for neuron in data_healthy:
    if neuron[neuron!=0].shape[0]>1000:
        list_healthy_neuron.append(neuron[neuron!=0])

## Run the model on the interval 100-300s

In [14]:
dataframe = pd.DataFrame()
counter = 0 
for neuron in list_healthy_neuron:
    neuron=neuron[neuron>100*10000]
    neuron=neuron[neuron<300*10000]
    ISI_healthy = np.diff(neuron)/10000
    map_estimate = Bayesian_mixture_model(ISI_healthy)
    df = pd.DataFrame.from_dict(map_estimate,orient='index')
    dataframe = pd.concat([dataframe,df],axis = 1)
    counter+=1

Multiprocess sampling (6 chains in 6 jobs)
CompoundStep
>Metropolis: [w]
>Metropolis: [sigma3]
>Metropolis: [mu3]
>Metropolis: [sigma]
>Metropolis: [mu2]
>Metropolis: [lam1]
>Metropolis: [mu1]
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.
logp = 11,847, ||grad|| = 0.052797: 100%|██████████| 40/40 [00:00<00:00, 631.28it/s]  
/home/riccardo_levi_25/anaconda3/lib/python3.7/site-packages/pymc3/sampling.py:1247: UserWarning: samples parameter is smaller than nchains times ndraws, some draws and/or chains may not be represented in the returned posterior predictive sample
  "samples parameter is smaller than nchains times ndraws, some draws "


P_value:  0.14494513582437837


Multiprocess sampling (6 chains in 6 jobs)
CompoundStep
>Metropolis: [w]
>Metropolis: [sigma3]
>Metropolis: [mu3]
>Metropolis: [sigma]
>Metropolis: [mu2]
>Metropolis: [lam1]
>Metropolis: [mu1]
The rhat statistic is larger than 1.4 for some parameters. The sampler did not converge.
The estimated number of effective samples is smaller than 200 for some parameters.
logp = 12,341, ||grad|| = 0.024489: 100%|██████████| 63/63 [00:00<00:00, 628.38it/s]  
/home/riccardo_levi_25/anaconda3/lib/python3.7/site-packages/pymc3/sampling.py:1247: UserWarning: samples parameter is smaller than nchains times ndraws, some draws and/or chains may not be represented in the returned posterior predictive sample
  "samples parameter is smaller than nchains times ndraws, some draws "


P_value:  0.001239986706637319


Multiprocess sampling (6 chains in 6 jobs)
CompoundStep
>Metropolis: [w]
>Metropolis: [sigma3]
>Metropolis: [mu3]
>Metropolis: [sigma]
>Metropolis: [mu2]
>Metropolis: [lam1]
>Metropolis: [mu1]


KeyboardInterrupt: 